# Dataloader

In [1]:
from core.data.nuscenes import NuScenesDataset

In [3]:
NU = NuScenesDataset('core/data/datasets/nuscenes',
                               0.05, split='mini', task='train')

In [4]:
item = NU.__getitem__(0)

In [5]:
item.keys()

dict_keys(['lidar', 'targets', 'targets_mapped', 'inverse_map'])

In [6]:
item['lidar'].C.shape

torch.Size([26507, 4])

# Loading Network Parameters

In [7]:
import json
import torch
from core.model import SPVStd, SPVRnn

net_id = '@20'

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda:0'
    
print(device)

cuda:0


In [10]:
#Load config
net_config = json.load(open('core/pretrained/net{}.config'.format(net_id)))

In [9]:
net_config['num_classes'] = 17

In [11]:
#Load parameters
params = torch.load('core/pretrained/init{}'.format(net_id),
                 map_location=device)['model']

In [6]:
print(params['classifier.linear.weight'].requires_grad)
print(params['upsample.0.transition.net.bn.weight'].requires_grad)

False
False


In [27]:
del params['classifier.linear.weight']
del params['classifier.linear.bias']
params.keys()

odict_keys(['stem.0.kernel', 'stem.1.weight', 'stem.1.bias', 'stem.1.running_mean', 'stem.1.running_var', 'stem.1.num_batches_tracked', 'stem.3.kernel', 'stem.4.weight', 'stem.4.bias', 'stem.4.running_mean', 'stem.4.running_var', 'stem.4.num_batches_tracked', 'downsample.0.transition.net.conv.kernel', 'downsample.0.transition.net.bn.weight', 'downsample.0.transition.net.bn.bias', 'downsample.0.transition.net.bn.running_mean', 'downsample.0.transition.net.bn.running_var', 'downsample.0.transition.net.bn.num_batches_tracked', 'downsample.0.feature.0.net.0.net.conv.kernel', 'downsample.0.feature.0.net.0.net.bn.weight', 'downsample.0.feature.0.net.0.net.bn.bias', 'downsample.0.feature.0.net.0.net.bn.running_mean', 'downsample.0.feature.0.net.0.net.bn.running_var', 'downsample.0.feature.0.net.0.net.bn.num_batches_tracked', 'downsample.0.feature.0.net.1.net.conv.kernel', 'downsample.0.feature.0.net.1.net.bn.weight', 'downsample.0.feature.0.net.1.net.bn.bias', 'downsample.0.feature.0.net.1.ne

# SPVNAS Model

In [12]:
#Build the model
model = SPVStd(net_config, params, device)

In [32]:
for name, param in model.named_parameters():
                if name not in ['classifier.linear.weight', 'classifier.linear.bias']:
                    param.requires_grad = False
                
for name, param in model.named_parameters():
    if param.requires_grad == True:
        print(name)
        print(param.shape)

classifier.linear.weight
torch.Size([17, 128])
classifier.linear.bias
torch.Size([17])


With a random input vector.

In [9]:
from torchsparse import SparseTensor

sample_feat = torch.randn(1000, 4)
sample_coord = torch.randn(1000, 4).random_(997)
sample_coord[:, -1] = 0
        
x = SparseTensor(sample_feat, sample_coord.int()).to(device)

with torch.no_grad():
    print(model.forward(x))

tensor([[ 0.8170, -1.9681, -2.0076,  ...,  0.6973, -0.8072, -2.0817],
        [ 1.0878, -1.8475, -1.1225,  ...,  1.9824, -1.1658, -2.4323],
        [-3.2777, -1.4396, -2.9076,  ...,  0.4465,  1.9437, -2.5443],
        ...,
        [-2.5257, -2.6721, -2.3396,  ...,  4.2012,  0.1960, -2.4443],
        [-1.0198, -1.8608, -1.8722,  ...,  2.4786, -0.8197, -1.2147],
        [-0.3618, -2.2857, -2.6285,  ...,  4.3616, -1.0914, -2.6978]],
       device='cuda:0')


In [10]:
print(type(x.F))
sample_feat = torch.randn(1000, 4)
sample_coord = torch.randn(1000, 4).random_(997)
sample_coord[:, -1] = 0
        
x = SparseTensor(sample_feat, sample_coord.int())
print(type(x.F))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


With an actual input

In [13]:
item['lidar'].C[:, -1] = 0 #Set the batch dimension to 0

In [14]:
model.eval()
with torch.no_grad():
    x = item['lidar'].to(device)
    y = model.forward(x)

In [15]:
y.shape

torch.Size([26507, 19])

In [16]:
from core.data.mappings import *

In [17]:
p = torch.nn.functional.softmax(y[0], dim=0)
print(p)
print(torch.max(p))
print(torch.argmax(p))

tensor([0.0078, 0.0024, 0.0024, 0.0011, 0.0055, 0.0028, 0.0013, 0.0016, 0.0290,
        0.0043, 0.0353, 0.0260, 0.4114, 0.0291, 0.2651, 0.0043, 0.1681, 0.0019,
        0.0007], device='cuda:0')
tensor(0.4114, device='cuda:0')
tensor(12, device='cuda:0')


In [18]:
item['targets'].F[0]

tensor(5, dtype=torch.uint8)

The first dimension matches which is good, also 19 is for the 19 classes they keep in SPVNAS. It seems that softmax isn't a part of their model? Also it is hard to tell how the predictions are matching up with everything.

# SPVRnn

In [5]:
#Build the model
model = SPVRnn(net_config, params, device)

In [6]:
from torchsparse import SparseTensor

sample_feat = torch.randn(1000, 4)
sample_coord = torch.randn(1000, 4).random_(997)
sample_coord[:, -1] = 0
        
x = SparseTensor(sample_feat, sample_coord.int()).to(device)

In [7]:
with torch.no_grad():
#     x = item['lidar'].to(device)
    y = model.forward(x)

In [9]:
print(type(y))
print(y.shape)

<class 'torchsparse.point_tensor.PointTensor'>
torch.Size([1000, 48])


In [11]:
y.C[0]

tensor([   0., 1241.,  162.,  560.], device='cuda:0')

In [12]:
x.C[0]

tensor([   0, 1241,  162,  560], device='cuda:0', dtype=torch.int32)

# Sparse Convolution

In [11]:
import torchsparse.nn as spnn
from torchsparse import SparseTensor

In [18]:
sample_feat = torch.randn(1000, 4)
sample_coord = torch.randn(1000, 4).random_(997)
sample_coord[:, -1] = 0
        
x = SparseTensor(sample_feat, sample_coord.int()).to(device)

In [19]:
conv = spnn.Conv3d(4, 32, kernel_size=3, stride=1).to(device)
y = conv.forward(x)
print(y.F.shape, y.C.shape)

torch.Size([1000, 32]) torch.Size([1000, 4])
